With the help of our previous analysis, we have found out the following parameters are prefereed for the best model:  
`dense_layers = [0]
layer_sizes = [64]
conv_layers = [3]`

So lets run our model one more time (using the code in previous notebook) with the chosen parameters and save the model for further use

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
# more info on callbakcs: https://keras.io/callbacks/ model saver is cool too.
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time

pickle_in = open("data_model/X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("data_model/y.pickle","rb")
y = pickle.load(pickle_in)

X = X/255.0

dense_layers = [0]
layer_sizes = [64]
conv_layers = [3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)

            model = Sequential()

            model.add(Conv2D(layer_size, (3, 3), input_shape=X.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())

            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))

            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

            model.compile(loss='binary_crossentropy',
                          optimizer='adam',
                          metrics=['accuracy'],
                          )

            model.fit(X, y,
                      batch_size=32,
                      epochs=3,
                      validation_split=0.3,
                      callbacks=[tensorboard])

model.save('64x3-CNN.model')

3-conv-64-nodes-0-dense-1593928886
Epoch 1/3
546/546 [==============================] - 436s 799ms/step - loss: 0.6391 - accuracy: 0.6218 - val_loss: 0.5456 - val_accuracy: 0.7259
Epoch 2/3
546/546 [==============================] - 399s 731ms/step - loss: 0.5137 - accuracy: 0.7490 - val_loss: 0.4788 - val_accuracy: 0.7692
Epoch 3/3
546/546 [==============================] - 397s 726ms/step - loss: 0.4540 - accuracy: 0.7871 - val_loss: 0.4477 - val_accuracy: 0.7843
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: 64x3-CNN.model\assets


## Using the Model for Prediction

Lets start by preparing the data (as we did while training)

In [9]:
import cv2
import tensorflow as tf

CATEGORIES = ["Dog", "Cat"]

def prepare(filepath):
    IMG_SIZE = 100
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)  # read in the image, convert to grayscale
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize image to match model's expected sizing
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

Next, we want to load our model:

In [10]:
model = tf.keras.models.load_model("64x3-CNN.model")

Now, we can make a prediction:

In [49]:
# remember to pass a list of files for prediction
prediction = model.predict([prepare('data/predict_img/dog1.jpg')])  
prediction

array([[0.]], dtype=float32)

Lets print the classification

In [50]:
print(CATEGORIES[int(prediction[0][0])])

Cat
